In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import sklearn
import numpy as np
from tqdm import tqdm

In [13]:
# Actual Values from 2020 ESPN draft league
actual_data = pd.read_csv("./data/2020_espn_draft_results.csv", index_col=0)
accumulated_data = pd.read_csv("./data/fantasy_football_auction_values.csv")

In [15]:
actual_data.head()

,player_names,bid_amounts,keepers,round_num,round_pick
0,Chris Carson,15,True,1,1
1,Derrick Henry,27,True,1,2
2,Lamar Jackson,9,True,1,3
3,George Kittle,10,True,1,4
4,Terry McLaurin,5,True,1,5


In [16]:
accumulated_data.head()

,Player_Names,ESPN_Price,ESPN_Avg_Price,Yahoo_Price,Yahoo_Avg_Price,NFLprice,NFL_Avg_Price,Average_Price,Proj_Price,Inflated_Price
0,Christian McCaffrey,63,70,64,76,70,63,69,81,81
1,Saquon Barkley,60,61,62,71,67,56,63,72,72
2,Ezekiel Elliott,60,58,55,64,52,45,56,67,67
3,Michael Thomas,51,57,54,56,49,45,53,43,43
4,Dalvin Cook,59,51,54,58,50,37,49,57,57


In [20]:
'Chris Carson' in list(actual_data.player_names)

True

In [22]:
'Chris Carson' in list(accumulated_data.Player_Names)

True

In [24]:
data = pd.merge(actual_data, accumulated_data, left_on="player_names", right_on="Player_Names").drop("Player_Names", axis=1)

In [27]:
data.shape

(198, 14)

In [32]:
# Disregard datapoints where they're kept as Keepers.
data = data[data['keepers'] != True].drop("keepers", axis=1)

In [33]:
data.shape

(179, 13)

In [35]:
data.head()

,player_names,bid_amounts,round_num,round_pick,ESPN_Price,ESPN_Avg_Price,Yahoo_Price,Yahoo_Avg_Price,NFLprice,NFL_Avg_Price,Average_Price,Proj_Price,Inflated_Price
19,Travis Kelce,20,2,8,35,37,43,40,36,29,35,37,37
20,Christian McCaffrey,75,2,9,63,70,64,76,70,63,69,81,81
21,Zach Ertz,11,2,10,19,16,22,13,26,16,15,19,19
22,Michael Thomas,55,2,11,51,57,54,56,49,45,53,43,43
23,Aaron Jones,34,2,12,43,41,36,40,30,29,37,45,45


In [61]:
# Decision Trees
mse = []
mae = []
r2 = []
for i in tqdm(range(10)):
    xtrain, xtest, ytrain, ytest = train_test_split(data.drop(["player_names", "bid_amounts"], axis=1), data["bid_amounts"], test_size=0.20)
    model = DecisionTreeRegressor()
    model.fit(xtrain, ytrain)
    predictions = model.predict(xtest)
    mse.append(mean_squared_error(predictions, ytest))
    mae.append(mean_absolute_error(predictions, ytest))
    r2.append(r2_score(predictions, ytest))
print(f"MSE: {np.mean(mse)}")
print(f"MAE: {np.mean(mae)}")
print(f"R2: {np.mean(r2)}")

100%|██████████| 10/10 [00:00<00:00, 81.50it/s]

MSE: 43.519444444444446
MAE: 3.780555555555556
R2: 0.8325346537405565


In [64]:
# Random Forests
mse = []
mae = []
r2 = []
for i in tqdm(range(10)):
    xtrain, xtest, ytrain, ytest = train_test_split(data.drop(["player_names", "bid_amounts"], axis=1), data["bid_amounts"], test_size=0.20)
    model = RandomForestRegressor()
    model.fit(xtrain, ytrain)
    predictions = model.predict(xtest)
    mse.append(mean_squared_error(predictions, ytest))
    mae.append(mean_absolute_error(predictions, ytest))
    r2.append(r2_score(predictions, ytest))
print(f"MSE: {np.mean(mse)}")
print(f"MAE: {np.mean(mae)}")
print(f"R2: {np.mean(r2)}")

100%|██████████| 10/10 [00:03<00:00,  3.27it/s]

MSE: 17.47434638888889
MAE: 2.304194444444445
R2: 0.9185429820351197


In [67]:
# XGBoost
mse = []
mae = []
r2 = []
for i in tqdm(range(10)):
    xtrain, xtest, ytrain, ytest = train_test_split(data.drop(["player_names", "bid_amounts"], axis=1), data["bid_amounts"], test_size=0.20)
    model = XGBRegressor()
    model.fit(xtrain, ytrain)
    predictions = model.predict(xtest)
    mse.append(mean_squared_error(predictions, ytest))
    mae.append(mean_absolute_error(predictions, ytest))
    r2.append(r2_score(predictions, ytest))
print(f"MSE: {np.mean(mse)}")
print(f"MAE: {np.mean(mae)}")
print(f"R2: {np.mean(r2)}")

100%|██████████| 10/10 [00:00<00:00, 13.07it/s]

MSE: 18.513022842803082
MAE: 2.4549738384783266
R2: 0.9169380973804138
